<img src="images/TilePyramid.jpg" width=500 alt="Tile Pyramid"></img>

Image from [Web Map Tile Services for Spatial Data Infrastructures: Management and Optimization](https://www.intechopen.com/chapters/38302)

# Web Map Services

Web map services (WMS) provide small, pre-rendered image tiles that collectively form a larger map when displayed in a grid-like pattern. Each tile represents a specific geographic area and zoom level, allowing users to view maps at different scales and locations.

## Tile Specifications

Various specifications govern the functionality of web map services, including TMS (Tile Map Service), WMTS (Web Map Tile Service), and XYZ. These specifications define how the tiles are organized, accessed, and displayed within a web mapping context.

However, it is not necessary to delve into the technical details of these underlying specifications in order to benefit from web map services. The implementation takes care of these intricacies behind the scenes, allowing users to seamlessly utilize the service without requiring in-depth knowledge.

Nonetheless, it is useful to have an understanding of the format that each specification follows. Here are examples of URL patterns associated with each specification:

- TMS (Tile Map Service): http://example.com/tiles/{z}/{x}/{y}.png
- WMTS (Web Map Tile Service): http://example.com/wmts/{Layer}/{TileMatrixSet}/{TileMatrix}/{TileRow}/{TileCol}.png
- XYZ: http://example.com/{z}/{x}/{y}.png

These URLs represent the structure for accessing individual map tiles based on their zoom level ({z}), column ({x}), and row ({y}) coordinates, and sometimes, as you'll see later, it's based on the bounds {XMIN},{XMAX},{YMIN},{YMAX}. Other specifications exist as well, such as Quadkey used by Bing Maps and GeoPackage for storing tiles in a single file.

## Online WMS Viewers

Online WMS viewers provide a user-friendly interface for accessing and visualizing geospatial data through web map services. They offer a convenient way to explore and interact with various layers of information, ranging from weather data to satellite imagery.

Here are some examples of online WMS viewers:

NOAA Weather Viewer (https://viewer.weather.noaa.gov/): This viewer, provided by the National Oceanic and Atmospheric Administration (NOAA), allows users to access and visualize real-time weather information, including radar data, forecasts, and satellite imagery. It offers an intuitive interface for exploring weather conditions and monitoring meteorological phenomena.

National Map Viewer (https://apps.nationalmap.gov/viewer/): The National Map Viewer, developed by the United States Geological Survey (USGS), provides access to a wide range of geospatial data, including topographic maps, aerial imagery, and elevation data. Users can explore and overlay different datasets, enabling them to analyze and visualize various geographic features.

NASA Worldview (https://worldview.earthdata.nasa.gov/): NASA Worldview is a powerful online WMS viewer that offers access to a vast collection of satellite imagery and Earth observation data. Users can view imagery captured by different satellite missions and explore global environmental changes over time. The viewer supports advanced visualization and analysis tools, allowing users to study phenomena such as atmospheric conditions, land cover changes, and natural disasters.

While there are existing online WMS viewers that provide access to a wide range of geospatial data, **the true power of web map services (WMS) lies in the ability to overlay them with your own data.**

We will explore how to do this below.

## Prerequisites

The following packages are good to know, but not required, as we will be walking through them below.

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to GeoViews](https://geoviews.org/) | Helpful | |
| [Intro to Panel](https://panel.holoviz.org/) | Helpful | |
| [Intro to Requests](https://requests.readthedocs.io/en/latest/) | Helpful | |
| [Intro to Cartopy](https://foundations.projectpythia.org/core/cartopy/cartopy.html) | Helpful | |

- **Time to learn**: 10 minutes

---

## Imports

Let’s first import a few packages.

GeoViews is a Python library that facilitates the integration of WMS and other geospatial data sources with your own datasets. It provides a high-level interface for working with geographic data and simplifies the process of creating interactive visualizations.

CartoPy is a Python library that specializes in handling map projections and geospatial data visualization. It provides a wide range of tools and functionalities to work with geospatial datasets, making it easier to create maps with different projections.

Panel is a Python library that offers a set of flexible and powerful tools for creating interactive dashboards and apps. It allows you to build custom user interfaces with interactive controls, widgets, and layout components, enabling rich interactivity for your visualizations and data analysis workflows.

Requests is a popular Python library for making HTTP requests. It simplifies sending HTTP requests to web servers and handling the responses. In the context of geospatial data visualization, requests can be used to retrieve data from web services, such as WMS (Web Map Service) endpoints, to integrate external data sources into your visualizations.

The next line, `gv.extension("bokeh")`, enables the Bokeh (interactive) plotting backend for GeoViews. GeoViews supports multiple plotting backends, such as Bokeh and Matplotlib, which allow you to choose the one that best suits your needs.

Finally, `pn.extension()` initializes the panel library and sets up the necessary environment for creating interactive panels and dashboards. You may specify configurations like `sizing_mode="stretch_width"` within `pn.extension()`.

In [ ]:
import geoviews as gv
import cartopy.crs as ccrs
import panel as pn
import requests

gv.extension("bokeh")
pn.extension()

## Using Built-in Sources

Here's how to use GeoViews to access built-in WMS tile sources effortlessly.

By activating the "pan" and "wheel_zoom" tools, users gain interactive control over the map. They can easily navigate the map by dragging, exploring different areas, and experiencing a sense of flexibility and control.

To experience the capabilities of WMS and GeoViews, try zooming in on the map. As you zoom in, you will notice that more details and finer features become visible, providing a richer and more detailed representation of the geographic data.

To see all available built-in tile sources, please visit the [GeoViews documentation](https://geoviews.org/gallery/bokeh/tile_sources.html).

In [ ]:
basemap = gv.tile_sources.OSM().opts(active_tools=["pan", "wheel_zoom"])
basemap

## Overlaying Custom Data

By integrating WMS layers with your own data, you can unlock the potential of geospatial information and create maps that communicate intricate spatial relationships, highlight patterns and trends, and offer valuable insights.

Let's illustrate this with an example. In isolation, the coordinates (40.1N, -88.26W) may not hold much significance. However, when overlaying the WMS tiles and zooming in, you might discover that these coordinates correspond to the location of the University of Illinois at Urbana-Champaign, the author's alma mater. This simple overlay demonstrates how WMS layers can provide context and meaningful information to otherwise isolated data points.

Note that `crs` and `projection` were specified.

The `crs=ccrs.PlateCarree()` argument specifies that the data coordinates are **from** the Plate Carree coordinate reference system, which is commonly used for latitude and longitude values.

On the other hand, the `projection` parameter determines what the data should be transformed **to** before being displayed.

**At the time of writing, GeoViews only supports overlaying data with tile sources when the map's projection is set to `ccrs.GOOGLE_MERCATOR`.**

Lastly, the global_extent=True option ensures that the map displays the entire world.

In [ ]:
coord_point = gv.Points((-88.24, 40.1), crs=ccrs.PlateCarree()).opts(projection=ccrs.GOOGLE_MERCATOR, global_extent=True)

basemap * coord_point

## Accessing External Sources

Although the availability of built-in sources is convenient, it may not always be sufficient for all use cases. However, the good news is that there are numerous external sources available, such as NOAA, which can be easily accessed, and depending on the source, free of charge!

For example, if you want to access radar base reflectivity provided by NOAA, you can create a `gv.WMTS` element with the appropriate URL and parameters specific to the external tile service.

In [ ]:
radar_url_fmt = """
https://idpgis.ncep.noaa.gov/arcgis/services/radar/radar_base_reflectivity_time/ImageServer/WMSServer?
SERVICE=WMS
&VERSION=1.3.0
&REQUEST=GetMap
&FORMAT=image/png
&TRANSPARENT=true
&WIDTH=256
&HEIGHT=256
&CRS=EPSG:3857
&BBOX={XMIN},{YMIN},{XMAX},{YMAX}
&LAYERS=0
"""

radar_imagery = gv.WMTS(radar_url_fmt, crs=ccrs.GOOGLE_MERCATOR).opts(projection=ccrs.GOOGLE_MERCATOR, global_extent=True)

basemap * radar_imagery

## Building WMS URLs

You might be wondering where that URL came from.

Admittedly, it's not very straightforward, but by following these instructions, it becomes quite simple!

1. Begin by searching "NOAA WMS" on Google.
2. Locate and click on the first search result, which is "[WebServices](https://www.weather.gov/gis/WebServices)".
3. Choose a specific link, such as "[Base Reflectivity Radar](https://idpgis.ncep.noaa.gov/arcgis/rest/services/NWS_Observations/radar_base_reflectivity/MapServer)".
4. On the top left of the page, click on the small text labeled "[WMS](https://idpgis.ncep.noaa.gov/arcgis/services/NWS_Observations/radar_base_reflectivity/MapServer/WMSServer?request=GetCapabilities&service=WMS)".
5. Remove the section "request=GetCapabilities&" from the URL.
6. Use the following prompt for ChatGPT: "Create a valid WMS URL by utilizing EPSG:3857 with {XMIN}, {YMIN}... to format the URL https://idpgis.ncep.noaa.gov/arcgis/services/NWS_Observations/radar_base_reflectivity/MapServer/WMSServer?service=WMS".
7. By following these steps, you will obtain a URL that is ready to be used with `gv.WMTS`!

In [ ]:
gv.WMTS("https://idpgis.ncep.noaa.gov/arcgis/services/NWS_Observations/radar_base_reflectivity/MapServer/WMSServer?service=WMS&version=1.3.0&request=GetMap&layers=0&styles=default&format=image/png&transparent=true&crs=EPSG:3857&bbox={XMIN},{YMIN},{XMAX},{YMAX}&width=512&height=512")

## Troubleshooting URLs

If you encounter a blank page as a result, try formatting the URL using these XMIN, YMIN, XMAX, YMAX values, which represent the bounds of the world in Mercator coordinates.

If it returns a white canvas, it probably means that there aren't any issues--there's simply no ongoing event, e.g. no precipitation for the radar to pick up on.

If you receive a detailed error message, that's good news--you can make the necessary revisions.

Here's an example of a URL missing a parameter.

In [ ]:
url = """
https://idpgis.ncep.noaa.gov/arcgis/services/NWS_Climate_Outlooks/cpc_6_10_day_outlk/MapServer/WMSServer?
SERVICE=WMS
&VERSION=1.3.0
&REQUEST=GetMap
&FORMAT=image/png
&TRANSPARENT=true
&WIDTH=256
&HEIGHT=256
&CRS=EPSG:3857
&BBOX={XMIN},{YMIN},{XMAX},{YMAX}
&LAYERS=0
""".format(XMIN=-20037507.539400,YMIN=1638517.444800,XMAX=20037260.918700,YMAX=7714669.394600).replace("\n", "")

resp = requests.get(url)
print(resp.text)

To address this issue, you can either review the documentation or simply input the URL along with the error into ChatGPT (the latter is usually easier).

Here's the revised URL; notice the response is no longer the same.

In [ ]:
url = """
https://idpgis.ncep.noaa.gov/arcgis/services/NWS_Climate_Outlooks/cpc_6_10_day_outlk/MapServer/WMSServer?
SERVICE=WMS
&VERSION=1.3.0
&REQUEST=GetMap
&FORMAT=image/png
&TRANSPARENT=true
&WIDTH=256
&HEIGHT=256
&CRS=EPSG:3857
&BBOX={XMIN},{YMIN},{XMAX},{YMAX}
&LAYERS=0
&STYLES=
""".format(XMIN=-20037507.539400,YMIN=1638517.444800,XMAX=20037260.918700,YMAX=7714669.394600).replace("\n", "")

resp = requests.get(url)
print(resp.content)

Upon re-plugging the modified URL into `gv.WMTS` without the formatting, we see a map of the US!

In [ ]:
outlook_url_fmt = """
https://idpgis.ncep.noaa.gov/arcgis/services/NWS_Climate_Outlooks/cpc_6_10_day_outlk/MapServer/WMSServer?
SERVICE=WMS
&VERSION=1.3.0
&REQUEST=GetMap
&FORMAT=image/png
&TRANSPARENT=true
&WIDTH=256
&HEIGHT=256
&CRS=EPSG:3857
&BBOX={XMIN},{YMIN},{XMAX},{YMAX}
&LAYERS=0
&STYLES=
"""

outlook_imagery = gv.WMTS(outlook_url_fmt)
outlook_imagery

## Getting Legends

A map loses its usefulness without a legend, unless the colors or symbols used are widely recognized.

As an example, let's retrieve the legend for the CPC 6-10 Days Outlook map.

To begin, follow step 4 mentioned earlier to navigate to the "GetCapabilities" page.

Next, "Find" (CTRL + F) the term "legend" on the page. This will help you locate the corresponding URL of the legend image.

Once you have the legend image URL, utilize `requests` to fetch the image data. Then, leverage the capabilities of Panel to display the legend image alongside the map in a visually appealing manner.

In [ ]:
legend_url = "https://idpgis.ncep.noaa.gov/arcgis/services/NWS_Climate_Outlooks/cpc_6_10_day_outlk/MapServer/WmsServer?request=GetLegendGraphic%26version=1.3.0%26format=image/png%26layer=0"
legend_image = pn.pane.Image(requests.get(legend_url).content)

pn.Row(
    basemap * outlook_imagery,
    legend_image
)

## Summary

Web map services (WMS) provide pre-rendered map tiles for displaying geospatial data.

They can be accessed through URLs based on specifications like TMS, WMTS, and XYZ. Online WMS viewers offer user-friendly interfaces for exploring geospatial data.

However, the true power of WMS lies in overlaying them with your own data. Packages like GeoViews, Panel, Requests, and Cartopy facilitate this integration.

For more examples of how to use WMS, check out the example workflows.

Next, we will explore how to use web feature services.